# Ejercicio 5 - Algoritmo VQE


## Contexto histórico

Durante la última década, las computadoras cuánticas han madurado rápidamente y han comenzado a hacer realidad el sueño inicial de Feynman de tener un sistema de cómputo que pudiese simular las leyes de la naturaleza de manera cuántica. Un artículo de 2014 con Alberto Peruzzo como primer autor introdujo el **algoritmo VQE** (de sus iniciales en inglés **Variational Quantum Eigensolver**, o *Solución cuántica variacional al problema de autoestados y autovalores*), un algoritmo con la intención de conseguir el estado fundamental (de mínima energía) de una molécula, con circuitos mucho más superficiales que los de otros enfoques. [1] Y, en 2017, el equipo IBM Quantum utilizó el algoritmo VQE para simular la energía fundamental de una molécula de hidruro de litio.[2]

La magia del algoritmo VQE viene dejarle a cargo una parte del esfuerzo de procesar el problema a una computadora clásica. El algoritmo comienza con un circuito cuántico parametrizado llamado ansatz (la conjetura) que busca el parámetro óptimo para este circuito usando un optimizador clásico. La ventaja del algoritmo VQE sobre los algoritmos clásicos viene del hecho de que una unidad cuántica de procesamiento puede representar y almacenar la función de onda exacta del problema, un problema exponencialmente difícil para una computadora clásica.

Este ejercicio 5 te permitirá a ti mismo hacer realidad el sueño de Feynman armando un VQE para determinar el estado fundamental y la energía de una molécula. Esto es interesante porque el estado fundamental puede ser utilizado para calcular varias propiedades moleculares, por ejemplo, las fuerzas exactas sobre los núcleos pueden servir para ejecutar simulaciones de dinámica molecular para explorar cómo evoluciona un sistema químico en el tiempo.[3]




### Referencias

1. Peruzzo, Alberto, et al. "A variational eigenvalue solver on a photonic quantum processor." Nature communications 5.1 (2014): 1-7.
2. Kandala, Abhinav, et al. "Hardware-efficient variational quantum eigensolver for small molecules and quantum magnets." Nature 549.7671 (2017): 242-246.
3. Sokolov, Igor O., et al. "Microcanonical and finite-temperature ab initio molecular dynamics simulations on quantum computers." Physical Review Research 3.1 (2021): 013125.

## Introducción
Para la implementación del VQE, tú deberás ser capaz de tomar decisiones de cómo quieres componer tu simulación, en particular concentrándote en los circuitos cuánticos del ansatz.
El motivo de esto es el hecho de que una de las tareas importantes cuando se ejecuta el algoritmo VQE en computadoras cuánticas ruidosas es la de reducir la pérdida de fidelidad (que introduce errores) encontrando el circuito cuántico más compacto posible capaz de representar el estado fundamental.
De manera práctica, esto conlleva a minimizar el número de compuertas a dos qubits (e.g. CNOTS) sin perder precisión.

<div class="alert alert-block alert-success">

<b>Objetivo</b> 

Encontrar los circuitos del ansatz para representar con precisión el estado fundamental de los problemas dados. ¡Sé creativo!
    
<b>Plan</b> 

Primero aprenderás cómo componer una simulación VQE para la molécula más pequeña y luego aplica lo que aprendiste para el caso de una más grande.

    
**1. Tutorial -  VQE para H$_2$:** familiarízate con VQE y escoge la mejor combinación de ansatz/optimizadores clásicos ejecutando las simulaciones de autovectores.

**2. Desafío final - VQE para LiH:** desempeña una investigación similar a la primera parte pero limitada al simulador de autovectores solamente. Emplea los esquemas de reducción del número de qubits disponibles en Qiskit y encuentra el circuito óptimo para este sistema más grande. Optimiza el circuito y utiliza tu imaginación para encontrar maneras de escoger los mejores componentes esenciales de los circuitos parametrizados y ensamblarlos para construir el circuito de un ansatz lo más compacto posible para el estado fundamental, incluso mejor que los que ya están disponibles en Qiskit.

</div>


<div class="alert alert-block alert-danger">

A continuación, hay una introducción a la teoría detrás de las simulaciones VQE. No tienes que entender todo antes de continuar. ¡No tengas miedo!


</div>



## Teoría

Aquí representamos el diagrama de flujo sobre cómo se llevan a cabo las simulaciones VQE en las computadoras cuánticas.


<img src="resources/workflow.png" width=800 height= 1400/>

La idea de base del enfoque híbrido cuántico-clásico es dejar a cargo al **CPU (en inglés classical processing unit, unidad clásica de procesamiento)** y al **QPU (unidad cuántica de procesamiento)** de las partes que saben hacer mejor. La CPU se ocupa de enumerar los términos que hay que medir para calcular la energía y también de optimizar los parámetros del circuito. La QPU implementa un circuito cuántico que representa el estado cuántico del sistema y mide la energía. Más detalles a continuación: 

**CPU** calcula eficientemente las energías asociadas a los brincos electrónicos (integrales a un/dos cuerpo/s por medios de el método de Hartree-Fock) que representan el hamiltoniano, el operador energía total. El [método de Hartree-Fock (HF)](https://es.wikipedia.org/wiki/M%C3%A9todo_de_Hartree-Fock) calcula eficientemente una función de onda aproximada del estado fundamental suponiendo que esta última puede ser representada por un solo determinante de Slater (e.g. para la molécula H$_2$ en la base STO-3G con 4 espín-orbitales y qubits, $|\Psi_{HF} \rangle = |0101 \rangle$ donde los electrones ocupan el espín-orbital de mínima energía). Lo que hace la QPU luego en el VQE es encontrar un estado cuántico (correspondiente al circuito y a sus parámetros) que puede representar también otros estados asociados a las correlaciones electrónicas faltantes (i.e.  estados $\sum_i c_i |i\rangle$ en  $|\Psi \rangle = c_{HF}|\Psi_{HF} \rangle + \sum_i c_i |i\rangle $ dónde $i$ es un código binario). 

Después del cálculo de HF, se le asigna a cada operador en el hamiltoniano una medición en la QPU usando transformaciones fermión-a-qubit (ver la sección Hamiltoniano más abajo). Uno puede analizar aún más las propiedades del sistema y reducir el número de qubits o recortar el circuito del ansatz:


- Para simetrías Z2 y reducción de dos qubits, ver [Bravyi *et al*, 2017](https://arxiv.org/abs/1701.08213v1).

- Para la forja de entrelazamiento (*entanglement forging*), ver [Eddins *et al.*, 2021](https://arxiv.org/abs/2104.10220v1).

- Para ansatz adaptativos, ver [Grimsley *et al.*,2018](https://arxiv.org/abs/1812.11173v2), [Rattew *et al.*,2019](https://arxiv.org/abs/1910.09694), [Tang *et al.*,2019](https://arxiv.org/abs/1911.10205). Puedes utilizar ideas de estos artículos para encontrar maneras de recortar los circuitos cuánticos.

**QPU** implementa circuitos cuánticos (ver la sección Ansatz más abajo), parametrizados por ángulos $\vec\theta$, que representan la función de onda del estado fundamental al colocar varias rotaciones de un qubit y entrelazadores (e.g. compuertas a dos qubits). La ventaja cuántica reposa sobre el hecho de que las QPU pueden eficientemente representar y almacenar la función de onda exacta, cosa que se vuelve intratable para sistemas de cómputo clásicos cuando hay más de unos cuantos átomos. Finalmente, la QPU mide los operadores escogidos (e.g. aquellos que representan el hamiltoniano).

A continuación, vamos un poco más en los detalles matemáticos de cada componente del algoritmo VQE. También puede ser de ayuda ver nuestro video [episodio en video sobre VQE](https://www.youtube.com/watch?v=Z-A6G0WVI9w).



### Hamiltoniano 
Aquí explicamos cómo obtener los operadores que necesitamos medir para obtener la energía de un sistema dado.
Estos términos están incluidos en el hamiltoniano molecular definido así:

$$
\begin{aligned}
\hat{H} &=\sum_{r s} h_{r s} \hat{a}_{r}^{\dagger} \hat{a}_{s} \\
&+\frac{1}{2} \sum_{p q r s} g_{p q r s} \hat{a}_{p}^{\dagger} \hat{a}_{q}^{\dagger} \hat{a}_{r} \hat{a}_{s}+E_{N N}
\end{aligned}
$$
con
$$
h_{p q}=\int \phi_{p}^{*}(r)\left(-\frac{1}{2} \nabla^{2}-\sum_{I} \frac{Z_{I}}{R_{I}-r}\right) \phi_{q}(r)
$$
$$
g_{p q r s}=\int \frac{\phi_{p}^{*}\left(r_{1}\right) \phi_{q}^{*}\left(r_{2}\right) \phi_{r}\left(r_{2}\right) \phi_{s}\left(r_{1}\right)}{\left|r_{1}-r_{2}\right|} 
$$

donde $h_{r s}$ y $g_{p q r s}$ son las integrales a uno/dos cuerpo/s (usando el método de Hartree-Fock) y  $E_{N N}$ la energía de repulsión nuclear.
La integral a un cuerpo representa la energía cinética de los electrones y su interacción con el núcleo.
Las integrales a dos cuerpos representan la interacción electrón-electrón.
Los operadores $\hat{a}_{r}^{\dagger}, \hat{a}_{r}$ representan la creación y destrucción de un electrón en el espín-orbital $r$ y requieren ser asignados a operadores, de manera que podamos medirlos en una computadora cuántica.
Nótese que el VQE minimiza la energía electrónica, así que tendrás que recuperar y añadir la energía de repulsión nuclear $E_{NN}$ para calcular la energía total.

Así que para cada elemento de matriz no nulo en los tensores $ h_{r s}$ y $g_{p q r s}$, podemos construir cadenas de Pauli (producto tensorial de operadores de Pauli) con la transformación fermión-a-qubit siguiente.
Por ejemplo, en la transformación de Jordan-Wigner para un orbital $r = 3$, obtenemos la siguiente cadena de Pauli:
$$
\hat a_{3}^{\dagger}= \hat \sigma_z \otimes \hat \sigma_z \otimes\left(\frac{ \hat \sigma_x-i \hat \sigma_y}{2}\right) \otimes 1 \otimes \cdots \otimes 1
$$

donde $\hat \sigma_x, \hat \sigma_y, \hat \sigma_z$ son los célebres operadores de Pauli. Los productos tensoriales de operadores $\hat \sigma_z$ se deben colocar para imponer las relaciones de anti-conmutación.
Aquí está una representación de la transformación Jordan-Wigner entre los 14 orbitales de la molécula de agua con 14 qubits:


<img src="resources/mapping.png" width=600 height= 1200/>

Entonces, uno simplemente reemplaza las excitaciones a uno/dos cuerpo/s (e.g. $\hat{a}_{r}^{\dagger} \hat{a}_{s}$, $\hat{a}_{p}^{\dagger} \hat{a}_{q}^{\dagger} \hat{a}_{r} \hat{a}_{s}$) en el hamiltoniano por las cadenas de Pauli correspondientes (i.e. $\hat{P}_i$, ver la imagen de arriba). La serie de operadores resultante está lista para ser medida con la QPU.
Para detalles adicionales ver [Seeley *et al.*, 2012](https://arxiv.org/abs/1208.5986v1).

### Ansatz

Hay dos tipos principales de ansatz que se pueden usar en problemas de química. 

- **Ansatz q-UCC** están inspirados en la física, y aproximadamente transforman las excitaciones electrónicas en circuitos cuánticos. Los ansatz q-UCCSD (`UCCSD`en Qiskit) contienen todas las posibles excitaciones a uno y dos electrones. El doble apareado q-pUCCD (`PUCCD`) y singlete q-UCCD0 (`SUCCD`) solo consideran subconjuntos de esas excitaciones (lo que implica circuitos significativamente más cortos) y han demostrado que proveen buenos resultados para perfiles de disociación. Por ejemplo, q-pUCCD no tiene excitaciones individuales y las excitaciones dobles se aparean como lo muestra la imagen de más abajo.

- **Ansatz heurístico (`TwoLocal`)** fueron creados para acortar la profundidad del circuito pero son todavía capaces de representar el estado fundamental.
En la figura de abajo, las compuertas R representan rotaciones parametrizadas de qubits individuales y $U_{CNOT}$ los entrelazadores (compuertas a dos qubits). La idea es que después de repetir un mismo bloque unas ciertas $D$-veces (con parámetros independientes) se puede alcanzar el estado fundamental.

Para detalles adicionales referirse a [Sokolov *et al.* (ansatz q-UCC)](https://arxiv.org/abs/1911.10864v2) y [Barkoutsos *et al.* (ansatz heurístico)](https://arxiv.org/pdf/1805.04340.pdf).

<img src="resources/ansatz.png" width=700 height= 1200/>



### VQE

Dado un operador hermítico $\hat H$ con $E_{min}$ un autovalor mínimo desconocido, asociado con el autoestado $|\psi_{min}\rangle$, VQE proporciona una estimación $E_{\theta}$ acotada por $E_{min}$:


\begin{align*}
    E_{min} \le E_{\theta} \equiv \langle \psi(\theta) |\hat H|\psi(\theta) \rangle
\end{align*}  

donde $|\psi(\theta)\rangle$ es el estado de ensayo asociado con $E_{\theta}$. Al aplicar un circuito parametrizado, representado por $U(\theta)$ a un estado inicial arbitrario $|\psi\rangle$, el algoritmo obtiene una estimación $U(\theta)|\psi\rangle \equiv |\psi(\theta)\rangle$ aplicada sobre $|\psi_{min}\rangle$. La estimación es optimizada de manera iterativa a través de optimizador clásico cambiando el parámetro $\theta$ y minimizando el valor esperado de $\langle \psi(\theta) |\hat H|\psi(\theta) \rangle$. 

Aplicaciones de VQE, son posibles en simulaciones de dinámica molecular, ver [Sokolov *et al.*, 2021](https://arxiv.org/abs/2008.08144v1), y cálculos de estados excitados, ver [Ollitrault *et al.*, 2019](https://arxiv.org/abs/1910.12890), solo por mencionar algunos casos.


<div class="alert alert-block alert-danger">
 
<b> Referencias para detalles adicionales</b> 

Para el tutorial de qiskit-nature que implementa este algoritmo ir [aquí](https://qiskit.org/documentation/nature/tutorials/01_electronic_structure.html) pero esto no será suficiente y puede que quieras buscar en la [primera página del repositorio de github](https://github.com/Qiskit/qiskit-nature) y la  [carpeta test](https://github.com/Qiskit/qiskit-nature/tree/main/test) que contiene casos de prueba que están escritos para cada componente, suministrando el código de base para cada funcionalidad.

</div>

## Parte 1: Tutorial - VQE para la molécula de H$_2$ 

En esta parte, harás una simulación de la molécula de H2 usando la base STO-3G junto con el driver PySCF y la transformación de Jordan-Wigner.
Te guiaremos a través de las secciones a continuación de manera que así podrás abordar luego problemas más difíciles.


#### 1. Driver

Las interfaces para acceder a los códigos clásicos de química en Qiskit se llaman drivers.
Tenemos por ejemplo `PSI4Driver`, `PyQuanteDriver` y `PySCFDriver` disponibles.

Al correr un driver (cálculo de Hartree-Fock para una serie de bases y de geometrías moleculares), en la celda siguiente, obtenemos toda la información necesaria sobre nuestra molécula para aplicar luego el algoritmo cuántico.


In [ ]:
from qiskit_nature.drivers import PySCFDriver

molecule = "H .0 .0 .0; H .0 .0 0.739"
driver = PySCFDriver(atom=molecule)
qmolecule = driver.run()

<div class="alert alert-block alert-danger">
    
<b> Tutorial 1 preguntas </b> 

Revisa los atributos de `qmolecule` y responde las preguntas siguientes.

1. Necesitamos saber cuáles son las características básicas de nuestra molécula. ¿Cuál es el número de electrones en tu sistema?
2. ¿Cuál es el número de orbitales moleculares? 
3. ¿Cuál es el número de espín-orbitales?
4. ¿Cuántos qubits son necesarios para simular esta molécula con la transformación de Jordan-Wigner?
5. ¿Cuál es el valor de la energía de repulsión nuclear?

Puedes encontrar las respuestas al final de este documento.
</div>

In [ ]:
# ESCRIBE TU CÓDIGO ENTRE ESTAS LÍNEAS - INICIO




# ESCRIBE TU CÓDIGO ENTRE ESTAS LÍNEAS - FIN

#### 2. Problema de la estructura electrónica

Puedes luego crear un objeto llamado `ElectronicStructureProblem` que puede producir la lista de operadores fermiónicos antes de transformarlos a qubits (cadenas de Pauli).


In [ ]:
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
problem = ElectronicStructureProblem(driver)

# Genera los operadores en segunda cuantización
second_q_ops = problem.second_q_ops()

# Hamiltoniano
main_op = second_q_ops[0]

#### 3. QubitConverter

Definamos la transformación que vas a utilizar en esta simulación. Puedes intentar una transformación diferente pero nos enfocaremos en la transformación de Jordan-Wigner o `JordanWignerMapper` ya que nos permite una correspondencia simple: 1 qubit representa un espín-orbital en la molécula.

In [ ]:
from qiskit_nature.mappers.second_quantization import ParityMapper, BravyiKitaevMapper, JordanWignerMapper
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter

# Establece la transformación y el qubitconverter
mapper_type = 'JordanWignerMapper'

if mapper_type == 'ParityMapper':
    mapper = ParityMapper()
elif mapper_type == 'JordanWignerMapper':
    mapper = JordanWignerMapper()
elif mapper_type == 'BravyiKitaevMapper':
    mapper = BravyiKitaevMapper()

converter = QubitConverter(mapper=mapper, two_qubit_reduction=False)

# Los operadores fermiónicos se transforman en operadores de qubits
num_particles = (problem.molecule_data_transformed.num_alpha,
             problem.molecule_data_transformed.num_beta)
qubit_op = converter.convert(main_op, num_particles=num_particles)

#### 4. Estado inicial
Como ya describimos en la sección Teoría, un buen estado inicial en química es el estado de HF (i.e. $|\Psi_{HF} \rangle = |0101 \rangle$). Lo podemos preparar de la siguiente manera:

In [ ]:
from qiskit_nature.circuit.library import HartreeFock

num_particles = (problem.molecule_data_transformed.num_alpha,
             problem.molecule_data_transformed.num_beta)
num_spin_orbitals = 2 * problem.molecule_data_transformed.num_molecular_orbitals
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)
print(init_state)

#### 5. Ansatz
Una de las decisiones más importantes es el circuito cuántico que vas a escoger para aproximar el estado fundamental.
Aquí hay un ejemplo de la biblioteca de circuitos de qiskit que contiene muchas posibilidades para componer tu propio circuito.


In [ ]:
from qiskit.circuit.library import TwoLocal
from qiskit_nature.circuit.library import UCCSD, PUCCD, SUCCD

# Escoge el ansatz
ansatz_type = "TwoLocal"

# Parámetros para el ansatz q-UCC
num_particles = (problem.molecule_data_transformed.num_alpha,
             problem.molecule_data_transformed.num_beta)
num_spin_orbitals = 2 * problem.molecule_data_transformed.num_molecular_orbitals

# Establece argumentos para 'twolocal'
if ansatz_type == "TwoLocal":
    # Rotaciones de un qubit que son aplicadas sobre todos los qubit con parámetros independientes
    rotation_blocks = ['ry', 'rz']
    # Compuertas entrelazantes
    entanglement_blocks = 'cx'
    # La manera en cómo están entrelazados los qubits
    entanglement = 'full'
    # Repeticiones de 'rotation_blocks + entanglement_blocks' con parámetros independientes
    repetitions = 3
    # Se salta la creación de la última capa de 'rotation_blocks'
    skip_final_rotation_layer = True
    ansatz = TwoLocal(qubit_op.num_qubits, rotation_blocks, entanglement_blocks, reps=repetitions, 
                      entanglement=entanglement, skip_final_rotation_layer=skip_final_rotation_layer)
    # Añade el estado inicial
    ansatz.compose(init_state, front=True, inplace=True)
elif ansatz_type == "UCCSD":
    ansatz = UCCSD(converter,num_particles,num_spin_orbitals,initial_state = init_state)
elif ansatz_type == "PUCCD":
    ansatz = PUCCD(converter,num_particles,num_spin_orbitals,initial_state = init_state)
elif ansatz_type == "SUCCD":
    ansatz = SUCCD(converter,num_particles,num_spin_orbitals,initial_state = init_state)
elif ansatz_type == "Custom":
    # Ejemplo de cómo escribir tu propio circuito
    from qiskit.circuit import Parameter, QuantumCircuit, QuantumRegister
    # Define el parámetro variacional
    theta = Parameter('a')
    n = qubit_op.num_qubits
    # Crea un circuito cuántico vacío
    qc = QuantumCircuit(qubit_op.num_qubits)
    qubit_label = 0
    # Coloca una compuerta Hadamard
    qc.h(qubit_label)
    # Colocal una escalera de CNOTs
    for i in range(n-1):
        qc.cx(i, i+1)
    # Separador visual
    qc.barrier()
    # Rotaciones rz sobre todos los qubits
    qc.rz(theta, range(n))
    ansatz = qc
    ansatz.compose(init_state, front=True, inplace=True)

print(ansatz)

#### 6. Backend
Aquí es donde especificas el simulador o el dispositivo donde quieres que se ejecute tu algoritmo.
Para este desafío, nos enfocaremos en el simulador `statevector_simulator`.


In [ ]:
from qiskit import Aer
backend = Aer.get_backend('statevector_simulator')

#### 7. Optimizador
El optimizador sirve de guía para la evolución de los parámetros del ansatz así que es importante investigar la convergencia de la energía ya que podría definir el número de medidas que tendrán que ser llevadas a cabo por la QPU.
Una desición inteligente podría reducir drásticamente el número de evaluaciones necesarias para obtener la energía. 

In [ ]:
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP

optimizer_type = 'COBYLA'

# Podrías querer ajustar los parámetros
# de cada optimizador, aquí están los valores usados por defecto
if optimizer_type == 'COBYLA':
    optimizer = COBYLA(maxiter=500)
elif optimizer_type == 'L_BFGS_B':
    optimizer = L_BFGS_B(maxfun=500)
elif optimizer_type == 'SPSA':
    optimizer = SPSA(maxiter=500)
elif optimizer_type == 'SLSQP':
    optimizer = SLSQP(maxiter=500)

#### 8. Solución exacta
Para propósitos educativos, podemos resolver exactamente el problema con una diagonalización exacta de la matriz hamiltoniana para saber hacia donde apuntar con el algoritmo VQE. 
Por supuesto, las dimesiones de la matriz crecen exponencialmente con el número de orbitales moleculares, puedes intentar hacer esto con una molécula grande para ver que tan lento se vuelve el proceso.
Para sistemas muy grandes, te quedarías sin memoria disponible intentando almacenar la función de onda.

In [ ]:
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
import numpy as np 

def exact_diagonalizer(problem, converter):
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

result_exact = exact_diagonalizer(problem, converter)
exact_energy = np.real(result_exact.eigenenergies[0])
print("Energía electrónica exacta", exact_energy)
print(result_exact)

# La energía electrónica de referencia para la molécula de H2 es -1.85336 Ha
# Verifícalo con el resultado tu VQE.

#### 9. VQE y los parámetros iniciales para el ansatz
Ahora podemos importar la clase VQE y ejecutar el algoritmo.

In [ ]:
from qiskit.algorithms import VQE
from IPython.display import display, clear_output

# Imprime y guarda los datos en listas
def callback(eval_count, parameters, mean, std):  
    # Sobrescribe la misma línea al imprimir
    display("Evaluación: {}, Energía: {}, Std: {}".format(eval_count, mean, std))
    clear_output(wait=True)
    counts.append(eval_count)
    values.append(mean)
    params.append(parameters)
    deviation.append(std)

counts = []
values = []
params = []
deviation = []

# Prepara los parámetros iniciales del ansatz
# Escogemos un pequeño paso fijo de desplazamiento 
# para que todos los participantes comiencen con un punto de partida similar
try:
    initial_point = [0.01] * len(ansatz.ordered_parameters)
except:
    initial_point = [0.01] * ansatz.num_parameters

algorithm = VQE(ansatz,
                optimizer=optimizer,
                quantum_instance=backend,
                callback=callback,
                initial_point=initial_point)

result = algorithm.compute_minimum_eigenvalue(qubit_op)

print(result)

#### 9. Puntaje
Necesitamos juzgar cuán buena es tu simulación VQE, el ansatz y optimizadores escogidos. 
Para ello, implementamos la siguiente función simple de puntaje:

$$ puntaje = N_{CNOT}$$

donde $N_{CNOT}$ es el número de CNOTs.
Sin embargo, tienes que alcanzar una precisión química de $\delta E_{chem} = 0.004$ mHa, que puede ser difícil de alcanzar dependiendo del problema.
Tienes que alcanzar la precisión que escogimos con un número mínimo de CNOTs para superar el desafío. ¡Mientras menor sea el puntaje, mejor!


In [ ]:
# Almacena el resultado en un diccionario
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller

# ''Unroller'' transpila tu circuito en CNOTs y compuertas U
pass_ = Unroller(['u', 'cx'])
pm = PassManager(pass_)
ansatz_tp = pm.run(ansatz)
cnots = ansatz_tp.count_ops()['cx']
score = cnots

accuracy_threshold = 4.0 # in mHa
energy = result.optimal_value

if ansatz_type == "TwoLocal":
    result_dict = {
        'optimizer': optimizer.__class__.__name__,
        'mapping': converter.mapper.__class__.__name__,
        'ansatz': ansatz.__class__.__name__,
        'rotation blocks': rotation_blocks,
        'entanglement_blocks': entanglement_blocks,
        'entanglement': entanglement,
        'repetitions': repetitions,
        'skip_final_rotation_layer': skip_final_rotation_layer,
        'energy (Ha)': energy,
        'error (mHa)': (energy-exact_energy)*1000,
        'pass': (energy-exact_energy)*1000 <= accuracy_threshold,
        '# of parameters': len(result.optimal_point),
        'final parameters': result.optimal_point,
        '# of evaluations': result.optimizer_evals,
        'optimizer time': result.optimizer_time,
        '# of qubits': int(qubit_op.num_qubits),
        '# of CNOTs': cnots,
        'score': score}
else:
    result_dict = {
        'optimizer': optimizer.__class__.__name__,
        'mapping': converter.mapper.__class__.__name__,
        'ansatz': ansatz.__class__.__name__,
        'rotation blocks': None,
        'entanglement_blocks': None,
        'entanglement': None,
        'repetitions': None,
        'skip_final_rotation_layer': None,
        'energy (Ha)': energy,
        'error (mHa)': (energy-exact_energy)*1000,
        'pass': (energy-exact_energy)*1000 <= accuracy_threshold,
        '# of parameters': len(result.optimal_point),
        'final parameters': result.optimal_point,
        '# of evaluations': result.optimizer_evals,
        'optimizer time': result.optimizer_time,
        '# of qubits': int(qubit_op.num_qubits),
        '# of CNOTs': cnots,
        'score': score}

# Gráfica los resultados
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1)
ax.set_xlabel('Iteraciones')
ax.set_ylabel('Energía')
ax.grid()
fig.text(0.7, 0.75, f'Energía: {result.optimal_value:.3f}\nPuntaje: {score:.0f}')
plt.title(f"{result_dict['optimizer']}-{result_dict['mapping']}\n{result_dict['ansatz']}")
ax.plot(counts, values)
ax.axhline(exact_energy, linestyle='--')
fig_title = f"\
{result_dict['optimizer']}-\
{result_dict['mapping']}-\
{result_dict['ansatz']}-\
Energy({result_dict['energy (Ha)']:.3f})-\
Score({result_dict['score']:.0f})\
.png"
fig.savefig(fig_title, dpi=300)

# Display and save the data
import pandas as pd
import os.path
filename = 'results_h2.csv'
if os.path.isfile(filename):
    result_df = pd.read_csv(filename)
    result_df = result_df.append([result_dict])
else:
    result_df = pd.DataFrame.from_dict([result_dict])
result_df.to_csv(filename)
result_df[['optimizer','ansatz', '# of qubits', '# of parameters','rotation blocks', 'entanglement_blocks',
    'entanglement', 'repetitions', 'error (mHa)', 'pass', 'score']]

<div class="alert alert-block alert-danger">
    
<b>Tutorial 2 preguntas</b> 

Experimenta con todos los parámetros y luego:

1. ¿Puedes conseguir tu mejor ansatz heurístico (mejor puntaje, modificando los parámetros del ansatz `TwoLocal`) y tu mejor optimizador?
2. ¿Puedes conseguir tu mejor ansatz q-UCC (escoge entre los ansatz `UCCSD, PUCCD or SUCCD`) y tu mejor optimizador?
3. En la celda donde definimos el ansatz ¿puedes modificar el ansatz `Custom` colocando tu mismo las compuertas para escribir un circuito mejor que tu circuito `TwoLocal`?

Para cada pregunta, provee los objetos `ansatz`. 
Recuerda, tienes que alcanzar la precisión química necesaria$|E_{exact} - E_{VQE}| \leq 0.004 $ Ha $= 4$ mHa.
    
</div>



In [ ]:
# ESCRIBE TU CÓDIGO ENTRE ESTAS LÍNEAS - INICIO




# ESCRIBE TU CÓDIGO ENTRE ESTAS LÍNEAS - FIN

## Parte 2: Desafío final - VQE para la molécula de LiH 

En esta parte, harás una simulación de la molécula de LiH usando la base STO-3G con el driver PySCF.

</div>
    
<div class="alert alert-block alert-success">

<b>Objetivo</b> 


Experimenta con todos los parámetros y luego encuentra el mejor ansatz. ¡Puedes ser tan creativo como quieras!

Para cada pregunta, provee los objetos `ansatz` como en la Parte 1. Tu puntaje final estará basado solamente en la Parte 2.
    
</div>
Ten en cuenta que el sistema ahora es más grande. Deduce cuántos qubits necesitarías para este sistema recuperando el número de espín-orbitales.


### Reduciendo el tamaño del problema

Tal vez querrás reducir el número de qubits de tu simulación:
- podrías congelar los electrones internos ya que no contribuyen significativamente en química y considerar solo los electrones de valencia. Esta funcionalidad ya está implementada en Qiskit. Así que inspecciona los diferentes transformadores en `qiskit_nature.transformers` y consigue aquel que implenta la aproximación de congelamiento interno.
- puedes usar `ParityMapper` junto con `two_qubit_reduction=True` para eliminar 2 qubits.
- puedes reducir el número de qubits si inspeccionas las simetrías de tu hamiltoniano. Busca una manera de utilizar `Z2Symmetries` de Qiskit.

###  Ansatz personalizado

Tal vez querrás explorar las ideas propuestas en [Grimsley *et al.*,2018](https://arxiv.org/abs/1812.11173v2), [H. L. Tang *et al.*,2019](https://arxiv.org/abs/1911.10205), [Rattew *et al.*,2019](https://arxiv.org/abs/1910.09694), [Tang *et al.*,2019](https://arxiv.org/abs/1911.10205).  Puedes incluso intentar algoritmos de aprendizaje automatizados (*machine learning*) para generar un mejor circuito ansatz.

### Montaje y la simulación

¡Ejecutemos ahora el circuito del cálculo de Hartree-Fock y el resto te lo dejamos a ti!

<div class="alert alert-block alert-danger">

<b>Atención</b> 

A continuación, proveemos el `driver`, el punto de partida `initial_point`, el estado inicial `initial_state` que deberán permanecer como se indica.
Eres libre entonces de explorar todo lo demás disponible en Qiskit.
Así que comienza con este punto de partida (todos los parámetros ajustados a  0.01):
`initial_point = [0.01] * len(ansatz.ordered_parameters)`
    o
`initial_point = [0.01] * ansatz.num_parameters`

y tu estado inicial tiene que ser el estado Hartree-Fock:
    
`init_state = HartreeFock(num_spin_orbitals, num_particles, converter)`
    
Para cada pregunta, provee un objeto tipo `ansatz`. Recuerda, tienes que alcanzar la precisión química necesaria$|E_{exact} - E_{VQE}| \leq 0.004 $ Ha $= 4$ mHa.

</div>

In [ ]:
from qiskit_nature.drivers import PySCFDriver

molecule = 'Li 0.0 0.0 0.0; H 0.0 0.0 1.5474'
driver = PySCFDriver(atom=molecule)
qmolecule = driver.run()

In [ ]:
# ESCRIBE TU CÓDIGO ENTRE ESTAS LÍNEAS - INICIO




# ESCRIBE TU CÓDIGO ENTRE ESTAS LÍNEAS - FIN

In [ ]:
# Verifica tu respuesta usando el código siguiente
from qc_grader import grade_ex5
freeze_core = False # cambiar a 'True' si congelaste los electrones internos
grade_ex5(ansatz,qubit_op,result,freeze_core)

In [ ]:
# Envía tu respuesta. Puedes volver a enviar tu respuesta en todo momento.
from qc_grader import submit_ex5
submit_ex5(ansatz,qubit_op,result,freeze_core)

## Respuestas de la parte 1

<div class="alert alert-block alert-danger">

<b>Preguntas</b> 
    
Revisa los atributos de `qmolecule` y responde las preguntas siguientes.

1. Necesitamos saber cuáles son las características básicas de nuestra molécula. ¿Cuál es el número de electrones en tu sistema?
2. ¿Cuál es el número de orbitales moleculares? 
3. ¿Cuál es el número de espín-orbitales?
4. ¿Cuántos qubits son necesarios para simular esta molécula con la transformación de Jordan-Wigner?
5. ¿Cuál es el valor de la energía de repulsión nuclear?
    
</div>

<div class="alert alert-block alert-success">

<b>Respuestas </b> 

1. `n_el = qmolecule.num_alpha + qmolecule.num_beta`
    
2. `n_mo = qmolecule.num_molecular_orbitals`
    
3. `n_so = 2 * qmolecule.num_molecular_orbitals`
    
4. `n_q = 2* qmolecule.num_molecular_orbitals`
    
5. `e_nn = qmolecule.nuclear_repulsion_energy`
    
    
</div>

## Additional information

**Created by:** Igor Sokolov, Junye Huang, Rahul Pratap Singh

**Latin American Spanish translation**: Mauricio Gómez Viloria, Manuel Morgado

**Version:** 1.0.1